# multiWavelength ptychography
 - This tutorial shows a reconstruction with a multiwavelength dataset measusred with a mouse brain sample, which is used in the publication: "L. Loetgering, X. Liu, A. C. C. De Beurs, M. Du, G. Kuijper, K. S. E. Eikema, and S. Witte, “Tailoring spatial entropy in extreme ultraviolet focused beams for multispectral ptychography,” Optica 8, 130 (2021)".
 - Download the dataset from figshare: https://figshare.com/articles/dataset/Brain_smoothBeam_poly_bin8_hdf5/19107068
 - A kind suggestion: check out general tutorials for CPM (conventional ptychographic microscopy) before diving into this one.

In [1]:
# %matplotlib notebook
%matplotlib qt
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import h5py

In [2]:
# import the fracPy module
import PtyLab
from PtyLab import ExperimentalData
from PtyLab import Reconstruction
from PtyLab import Monitor
from PtyLab import Params
from PtyLab import Engines

C:\ProgramData\Anaconda3\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


In [3]:
from PtyLab.utils.visualisation import hsvplot, hsvmodeplot, absplot, absmodeplot

In [4]:
# Step 1: Download the preprocessed data from surfdrive (https://figshare.com/articles/dataset/Brain_smoothBeam_poly_bin8_hdf5/19107068)

# set your data folder as fileFolder 
fileFolder = Path(r"D:\Du\FigshareData")
# select the hdf5 file
fileName = "Brain_smoothBeam_poly_bin8.hdf5"
# create the complete filePath
filePath = Path.joinpath(fileFolder,fileName)

In [5]:
# Step 2: Initialize the ExperimentalData class 

# Initialize the ExperimentalData using the filePath, and choose the operation Mode (default is 'CPM')
experimentalData = ExperimentalData(filePath, operationMode='CPM')
backgroundOffset = 10
experimentalData.entrancePupilDiameter = 0.2e-3
from PtyLab.utils.utils import posit
experimentalData.ptychogram = posit(experimentalData.ptychogram-backgroundOffset)
# show measured ptychogram in log scale. Need to close the window to continue
# experimentalData.showPtychogram()

"Unable to open object (object 'orientation' doesn't exist)"


In [6]:
# Set monitor properties
monitor = Monitor()
monitor.figureUpdateFrequency = 1
monitor.objectPlot = 'angle'  # complex abs angle
monitor.verboseLevel = 'high'  # high: plot two figures, low: plot only one figure
monitor.probeZoom = 1  # control probe plot FoV
monitor.objectZoom = 1   # control object plot FoV
monitor.objectPlotContrast = 8
monitor.probePlotContrast = 1

In [7]:
params = Params()
params.gpuSwitch = True
params.positionOrder = 'random'  # 'sequential' or 'random'
params.propagatorType = 'Fraunhofer'  # Fraunhofer Fresnel ASP scaledASP polychromeASP scaledPolychromeASP
params.probePowerCorrectionSwitch = False
params.modulusEnforcedProbeSwitch = True
params.comStabilizationSwitch = True
params.orthogonalizationSwitch = True
params.orthogonalizationFrequency = 5
params.intensityConstraint = 'standard'  # standard fluctuation exponential poission
params.couplingObjectSwitch = True
params.couplingObjectAleph = 0.5

In [9]:
# now, all our experimental data is loaded into experimental_data and we don't have to worry about it anymore.
# now create an object to hold everything we're eventually interested in
reconstruction = Reconstruction(experimentalData, params)
reconstruction.No = 2**10
reconstruction.nlambdaObject = 1
reconstruction.nlambdaProbe = len(reconstruction.spectralDensity)

# initialize probe and object and related params
reconstruction.initialProbe = 'circ'
reconstruction.initialObject = 'ones'
reconstruction.initializeObjectProbe()

# # customize initial probe quadratic phase
reconstruction.probe = reconstruction.probe*np.exp(1.j*2*np.pi/reconstruction.wavelength *
                                             (reconstruction.Xp**2+reconstruction.Yp**2)/(1*2e-2))

hsvmodeplot(np.squeeze(reconstruction.probe), pixelSize = reconstruction.dxp, axisUnit = 'mm')

Copying attribute wavelength
Copying attribute dxd
Copying attribute theta
Copying attribute spectralDensity
Copying attribute entrancePupilDiameter


In [12]:
# Run the reconstruction
## choose engine
# mPIE
multiPIE = Engines.multiPIE(reconstruction, experimentalData, params, monitor)
multiPIE.numIterations = 100
multiPIE.betaProbe = 0.25
multiPIE.betaObject = 0.25
multiPIE.reconstruct()

                                                                                                                       
iteration: 0                                                                                                           
error: 19.4                                                                                                            
estimated linear overlap: 60.9 %                                                                                       
estimated area overlap: 76.5 %                                                                                         
                                                                                                                       
iteration: 1                                                                                                           
error: 19.4                                                                                                            
estimated linear overlap: 60.8 %        

estimated linear overlap: 60.5 %                                                                                       
estimated area overlap: 74.8 %                                                                                         
                                                                                                                       
iteration: 14                                                                                                          
error: 19.2                                                                                                            
estimated linear overlap: 60.5 %                                                                                       
estimated area overlap: 75.0 %                                                                                         
                                                                                                                       
iteration: 15                           

iteration: 27                                                                                                          
error: 19.7                                                                                                            
estimated linear overlap: 60.4 %                                                                                       
estimated area overlap: 74.2 %                                                                                         
                                                                                                                       
iteration: 28                                                                                                          
error: 19.7                                                                                                            
estimated linear overlap: 60.5 %                                                                                       
estimated area overlap: 73.7 %          

estimated area overlap: 79.6 %                                                                                         
                                                                                                                       
iteration: 41                                                                                                          
error: 20.7                                                                                                            
estimated linear overlap: 60.5 %                                                                                       
estimated area overlap: 79.2 %                                                                                         
                                                                                                                       
iteration: 42                                                                                                          
error: 20.7                             

error: 21.3                                                                                                            
estimated linear overlap: 60.4 %                                                                                       
estimated area overlap: 78.6 %                                                                                         
                                                                                                                       
iteration: 55                                                                                                          
error: 21.4                                                                                                            
estimated linear overlap: 60.4 %                                                                                       
estimated area overlap: 78.3 %                                                                                         
                                        

                                                                                                                       
iteration: 68                                                                                                          
error: 21.8                                                                                                            
estimated linear overlap: 60.4 %                                                                                       
estimated area overlap: 77.3 %                                                                                         
                                                                                                                       
iteration: 69                                                                                                          
error: 21.9                                                                                                            
estimated linear overlap: 60.4 %        

estimated linear overlap: 60.4 %                                                                                       
estimated area overlap: 79.3 %                                                                                         
                                                                                                                       
iteration: 82                                                                                                          
error: 22.4                                                                                                            
estimated linear overlap: 60.4 %                                                                                       
estimated area overlap: 79.5 %                                                                                         
                                                                                                                       
iteration: 83                           

iteration: 95                                                                                                          
error: 22.7                                                                                                            
estimated linear overlap: 60.4 %                                                                                       
estimated area overlap: 78.9 %                                                                                         
                                                                                                                       
iteration: 96                                                                                                          
error: 23.0                                                                                                            
estimated linear overlap: 60.4 %                                                                                       
estimated area overlap: 79.2 %          

In [34]:
# now save the data
# reconstruction.saveResults('reconstruction.hdf5')